This code has been developed and ran on kaggle T4x2 gpus.

During data cleaning we have also turncated the texts to smaller size due to gpu contraints. as using max_length was giving us Cuda out of memory error. 

In [2]:
!pip install -U -q bitsandbytes accelerate transformers

In [3]:
import os
os.environ['HUGGINGFACE_TOKEN'] = 'huggingface-api-key'

In [4]:
from datasets import load_dataset
import re
from collections import Counter

#Load the dataset from Hugging Face
dataset = load_dataset("zqz979/meta-review")

Since data is in Dictionary format. I am converting it to pandas for easy implementation.

In [5]:
train = dataset['train'].to_pandas()
validation = dataset['validation'].to_pandas()
test = dataset['test'].to_pandas()

In [6]:
# Preview dataset structure
print(train.head())
print(validation.head())
print(test.head())

                                               Input  \

0  In this paper, the author investigates how to ...   

1  **Summary of contributions:** This paper propo...   

2  This paper addresses the problem of MoE routin...   

3  This paper discusses applications of variants ...   

4  The authors introduce the problem of telegraph...   



                                              Output  

0  This paper studies how to learn dexterous mani...  

1  This paper proposed a new family of losses for...  

2  Mixture-of-Expert (MoE) models have demonstrat...  

3  In this work, the authors conduct experiments ...  

4  This paper presents methods for telegraphic su...  

                                               Input  \

0  This paper presents an approach, called EstiNe...   

1  The paper aimed at improving the performance o...   

2  The submission shows the numerical instabiliti...   

3  This paper presents a method for training a ne...   

4  This paper proposes a new task o

In [7]:
# Load the dataset and convert to Pandas DataFrames, resetting the index immediately
train = dataset['train'].to_pandas().reset_index(drop=True)
validation = dataset['validation'].to_pandas().reset_index(drop=True)
test = dataset['test'].to_pandas().reset_index(drop=True)

# Now the DataFrames will no longer have an index column, so you won’t encounter it later during tokenization or conversion

# Preview the dataset structure after resetting the index
print(train.head())
print(validation.head())
print(test.head())


                                               Input  \

0  In this paper, the author investigates how to ...   

1  **Summary of contributions:** This paper propo...   

2  This paper addresses the problem of MoE routin...   

3  This paper discusses applications of variants ...   

4  The authors introduce the problem of telegraph...   



                                              Output  

0  This paper studies how to learn dexterous mani...  

1  This paper proposed a new family of losses for...  

2  Mixture-of-Expert (MoE) models have demonstrat...  

3  In this work, the authors conduct experiments ...  

4  This paper presents methods for telegraphic su...  

                                               Input  \

0  This paper presents an approach, called EstiNe...   

1  The paper aimed at improving the performance o...   

2  The submission shows the numerical instabiliti...   

3  This paper presents a method for training a ne...   

4  This paper proposes a new task o

# Preprocessing Steps

remove stopwords ( except of the latex symbols)
lowercase the text
tokenize the text
normalize space 
remove any special characters (except of the latex symbols)

In [8]:
import re
import nltk
import pandas as pd
from nltk.corpus import stopwords
from transformers import AutoTokenizer

# Download stop words from nltk
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

def is_latex_symbol(token):
    """Improved LaTeX symbol check for mathematical symbols."""
    latex_pattern = r"\\[a-zA-Z]+|[{}^]"
    return re.match(latex_pattern, token) is not None

# Preprocessing function tailored for your dataset
def preprocess_text(text):
    """Preprocesses the input/output text while preserving LaTeX symbols."""
    
    # Step 1: Lowercase the text
    text = text.lower() if text else ""
    
    # Step 2: Tokenize the text
    tokens = text.split()  # Tokenizing the text
    
    # Step 3: Remove stop words (except for LaTeX symbols)
    processed_tokens = []
    for token in tokens:
        if is_latex_symbol(token):  # Keep LaTeX symbols intact
            processed_tokens.append(token)
        elif token not in stop_words:  # Remove stop words only from the general text
            processed_tokens.append(token)
    
    # Step 4: Remove special characters from general text, but preserve LaTeX symbols
    processed_tokens = [re.sub(r'[^\w\\{}]', '', token) for token in processed_tokens]
    
    # Step 5: Normalize spaces and join tokens back into a single string
    processed_text = ' '.join(processed_tokens)
    
    return processed_text






[nltk_data] Downloading package stopwords to

[nltk_data]     /home1/kumararn/nltk_data...

[nltk_data]   Package stopwords is already up-to-date!


In [9]:
# Preprocessing step
train_cleaned = train.dropna(subset=['Input', 'Output'])
train_cleaned.loc[:, 'input_tokenized'] = train_cleaned['Input'].apply(preprocess_text)
train_cleaned.loc[:, 'output_tokenized'] = train_cleaned['Output'].apply(preprocess_text)

# Remove rows where input or output tokenized becomes empty
train_cleaned = train_cleaned[train_cleaned['input_tokenized'].str.strip() != '']
train_cleaned = train_cleaned[train_cleaned['output_tokenized'].str.strip() != '']

/tmp/SLURM_26327901/ipykernel_14193/214758001.py:3: SettingWithCopyWarning: 

A value is trying to be set on a copy of a slice from a DataFrame.

Try using .loc[row_indexer,col_indexer] = value instead



See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  train_cleaned.loc[:, 'input_tokenized'] = train_cleaned['Input'].apply(preprocess_text)

/tmp/SLURM_26327901/ipykernel_14193/214758001.py:4: SettingWithCopyWarning: 

A value is trying to be set on a copy of a slice from a DataFrame.

Try using .loc[row_indexer,col_indexer] = value instead



See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  train_cleaned.loc[:, 'output_tokenized'] = train_cleaned['Output'].apply(preprocess_text)


In [10]:
# Preprocessing step
validation_cleaned = validation.dropna(subset=['Input', 'Output'])
validation_cleaned.loc[:, 'input_tokenized'] = validation_cleaned['Input'].apply(preprocess_text)
validation_cleaned.loc[:, 'output_tokenized'] = validation_cleaned['Output'].apply(preprocess_text)

# Remove rows where input or output tokenized becomes empty
validation_cleaned = validation_cleaned[validation_cleaned['input_tokenized'].str.strip() != '']
validation_cleaned = validation_cleaned[validation_cleaned['output_tokenized'].str.strip() != '']


/tmp/SLURM_26327901/ipykernel_14193/3457451400.py:3: SettingWithCopyWarning: 

A value is trying to be set on a copy of a slice from a DataFrame.

Try using .loc[row_indexer,col_indexer] = value instead



See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  validation_cleaned.loc[:, 'input_tokenized'] = validation_cleaned['Input'].apply(preprocess_text)

/tmp/SLURM_26327901/ipykernel_14193/3457451400.py:4: SettingWithCopyWarning: 

A value is trying to be set on a copy of a slice from a DataFrame.

Try using .loc[row_indexer,col_indexer] = value instead



See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  validation_cleaned.loc[:, 'output_tokenized'] = validation_cleaned['Output'].apply(preprocess_text)


In [11]:
# Preprocessing step
test_cleaned = test.dropna(subset=['Input', 'Output'])
test_cleaned.loc[:, 'input_tokenized'] = test_cleaned['Input'].apply(preprocess_text)
test_cleaned.loc[:, 'output_tokenized'] = test_cleaned['Output'].apply(preprocess_text)

# Remove rows where input or output tokenized becomes empty
test_cleaned = test_cleaned[test_cleaned['input_tokenized'].str.strip() != '']
test_cleaned = test_cleaned[test_cleaned['output_tokenized'].str.strip() != '']


/tmp/SLURM_26327901/ipykernel_14193/78816890.py:3: SettingWithCopyWarning: 

A value is trying to be set on a copy of a slice from a DataFrame.

Try using .loc[row_indexer,col_indexer] = value instead



See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  test_cleaned.loc[:, 'input_tokenized'] = test_cleaned['Input'].apply(preprocess_text)

/tmp/SLURM_26327901/ipykernel_14193/78816890.py:4: SettingWithCopyWarning: 

A value is trying to be set on a copy of a slice from a DataFrame.

Try using .loc[row_indexer,col_indexer] = value instead



See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  test_cleaned.loc[:, 'output_tokenized'] = test_cleaned['Output'].apply(preprocess_text)


In [13]:


import gc
gc.collect()



683

I decided to use t5 small due to the memory constrains. The t5 model is a small text-to-text model which also support latex.

In [31]:
from transformers import AutoTokenizer
from datasets import Dataset
from transformers import AutoTokenizer

# Load the tokenizer for your model (e.g., T5)
tokenizer = AutoTokenizer.from_pretrained("google-t5/t5-small")
tokenizer.pad_token = tokenizer.eos_token

# Function to tokenize input and output
def tokenize_function(row):
    # Tokenize both input (meta-review) and output (summary)
    model_inputs = tokenizer(row['input_tokenized'], max_length= 1052, truncation=True, padding='max_length')
    
    # Tokenize output/labels (the summary)
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(row['output_tokenized'], max_length=1052, truncation=True, padding='max_length')
    
    # Return only required fields for training
    model_inputs['labels'] = labels['input_ids']
    return {
        'input_ids': model_inputs['input_ids'],
        'attention_mask': model_inputs['attention_mask'],
        'labels': model_inputs['labels']
    }

# Apply tokenization
train_tokenized = train_cleaned.apply(tokenize_function, axis=1, result_type='expand')
validation_tokenized = validation_cleaned.apply(tokenize_function, axis=1, result_type='expand')
test_tokenized = test_cleaned.apply(tokenize_function, axis=1, result_type='expand')

# Drop any unnecessary columns before converting to Hugging Face Dataset
train_tokenized = train_tokenized.drop(columns=['__index_level_0__'], errors='ignore')
validation_tokenized = validation_tokenized.drop(columns=['__index_level_0__'], errors='ignore')
test_tokenized = test_tokenized.drop(columns=['__index_level_0__'], errors='ignore')

# Convert to Hugging Face Dataset
train_dataset = Dataset.from_pandas(train_tokenized.reset_index(drop=True))
validation_dataset = Dataset.from_pandas(validation_tokenized.reset_index(drop=True))
test_dataset = Dataset.from_pandas(test_tokenized.reset_index(drop=True))


/home1/kumararn/ondemand/data/sys/research/test/test_venv/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:4109: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.

  warnings.warn(

/home1/kumararn/ondemand/data/sys/research/test/test_venv/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:4109: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.

  warnings.warn(

/home1/kumararn/ondemand/data/sys/research/test/test_venv/lib/python3.11/site-packages/transformers/to

In [13]:
train_dataset


Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 7680
})

In [14]:
import os
os.environ["WANDB_API_KEY"] ='Wandb api_key'

I used Wandb here to store my training logs but removed it during saving process.

In [15]:
#import wandb
# from fastai.callback.wandb import WandbCallback
# from kaggle_secrets import UserSecretsClient


In [17]:
# wandb.login() #only first time and not in commit

In [18]:
#wandb.init(project="quantization_summary")


In [19]:
from transformers import T5ForConditionalGeneration, Trainer, TrainingArguments,AutoModelForSeq2SeqLM

# Load the pre-trained T5 model
model = AutoModelForSeq2SeqLM.from_pretrained("google-t5/t5-small",device_map ='auto')

from transformers import TrainingArguments

# Set training arguments
training_args = TrainingArguments(
    output_dir="./results",           # output directory
    evaluation_strategy="epoch",      # evaluate each epoch
    per_device_train_batch_size=8,    # batch size for training
    per_device_eval_batch_size=8,     # batch size for evaluation
    num_train_epochs=11,               # number of training epochs
    weight_decay=0.01,                # strength of weight decay
    logging_dir="./logs",             # directory for storing logs
    remove_unused_columns=False ,
    learning_rate=5e-5,
    lr_scheduler_type="cosine"
    
)


# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset
)

# Start training
trainer.train()


/home1/kumararn/ondemand/data/sys/research/test/test_venv/lib/python3.11/site-packages/transformers/training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead

  warnings.warn(

Detected kernel version 3.10.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.

  5%|▍         | 500/10560 [03:50<1:17:33,  2.16it/s]

{'loss': 0.5959, 'grad_norm': 0.20634272694587708, 'learning_rate': 4.972392886566084e-05, 'epoch': 0.52}


                                                     

  9%|▉         | 960/10560 [08:03<1:13:21,  2.18it/s]

{'eval_loss': 0.5454779267311096, 'eval_runtime': 33.3852, 'eval_samples_per_second': 49.273, 'eval_steps_per_second': 6.17, 'epoch': 1.0}


  9%|▉         | 1000/10560 [08:22<1:13:38,  2.16it/s]

...

 99%|█████████▉| 10500/10560 [1:28:42<00:27,  2.16it/s]

{'loss': 0.5139, 'grad_norm': 0.17263339459896088, 'learning_rate': 3.982656874917945e-09, 'epoch': 10.94}


                                                       

100%|██████████| 10560/10560 [1:29:57<00:00,  1.96it/s]

{'eval_loss': 0.502321720123291, 'eval_runtime': 33.4274, 'eval_samples_per_second': 49.211, 'eval_steps_per_second': 6.163, 'epoch': 11.0}

{'train_runtime': 5397.0574, 'train_samples_per_second': 15.653, 'train_steps_per_second': 1.957, 'train_loss': 0.5178749055573435, 'epoch': 11.0}


TrainOutput(global_step=10560, training_loss=0.5178749055573435, metrics={'train_runtime': 5397.0574, 'train_samples_per_second': 15.653, 'train_steps_per_second': 1.957, 'total_flos': 2.349262989950976e+16, 'train_loss': 0.5178749055573435, 'epoch': 11.0})

Saved the model for future use

In [20]:
trainer.save_model(output_dir='./summarized_model')

# Calculating the Rogue values on test dataset without prompt

In [21]:
# Install the required library if not already installed
!pip install -q evaluate
!pip install rouge_score

from evaluate import load

# Load the ROUGE metric using the new 'evaluate' library
rouge_metric = load('rouge')

import torch

# Function to generate summaries from the model
def generate_summary(model, tokenizer, inputs, max_length=512):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    # Move the model to the correct device
    model.to(device)
    
    # Tokenize the inputs and move them to the same device as the model
    inputs = tokenizer(inputs, return_tensors="pt", truncation=True, padding="max_length", max_length=512)
    inputs = {key: value.to(device) for key, value in inputs.items()}  # Move input tensors to the correct device
    
    # Generate summary using the model
    summaries = model.generate(input_ids=inputs['input_ids'], attention_mask=inputs['attention_mask'], max_length=max_length)
    
    # Decode and return the summary
    return tokenizer.decode(summaries[0], skip_special_tokens=True)


# Evaluate model on the original validation data (before tokenization)
def evaluate_model_on_original(model, tokenizer, original_validation_dataset):
    references = []
    predictions = []
    
    for i in range(len(original_validation_dataset)):
        input_text = original_validation_dataset.iloc[i]['Input']  # Input meta-review
        reference_summary = original_validation_dataset.iloc[i]['Output']  # Ground truth summary
        
        # Generate summary from the model
        predicted_summary = generate_summary(model, tokenizer, input_text)
        
        # Store the reference and prediction
        references.append(reference_summary)
        predictions.append(predicted_summary)
    
    # Compute ROUGE scores
    results = rouge_metric.compute(predictions=predictions, references=references)
    return results

# Example usage:
# Assuming you have your fine-tuned model and tokenizer
results = evaluate_model_on_original(model, tokenizer, test_cleaned)

# Display the ROUGE scores
print(f"ROUGE-1: {results['rouge1']}")
print(f"ROUGE-2: {results['rouge2']}")
print(f"ROUGE-L: {results['rougeL']}")




huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...

To disable this warning, you can either:

	- Avoid using `tokenizers` before the fork if possible

	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...

To disable this warning, you can either:

	- Avoid using `tokenizers` before the fork if possible

	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)











ROUGE-1: 0.11874363474075633

ROUGE-2: 0.023526221252608955

ROUGE-L: 0.08437853235753011


In [28]:
!pip install -U huggingface_hub

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...

To disable this warning, you can either:

	- Avoid using `tokenizers` before the fork if possible

	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [25]:
from transformers import T5ForConditionalGeneration, Trainer, TrainingArguments,AutoModelForSeq2SeqLM

# Load the pre-trained T5 model
model = AutoModelForSeq2SeqLM.from_pretrained("./working/summarized_model/",device_map ='auto')


In [32]:
prompt = """ 
         You are an assistant which helps in summarizing the academic research-papers.You have to follow these rules while generating the response: 
         1. You have to understand the complete context of the input before generating summarization for that input.
         2. Any mathematical formulas in the input will be present in 'Latex' format.
         3. If you are including mathematical formula in your generation output you should also generate it in 'Latex' format.
         4. Some inputs have summary,strength,weakness,motivated,clarity,questions,significance,quality,originality,cons,however keywords which represents the author view on paper.
"""

In [33]:
test_cleaned

,Input,Output,input_tokenized,output_tokenized
0,This paper presents a bert-gat-crf framework ...,The paper present results using syntactic info...,paper presents bertgatcrf framework recognizin...,paper present results using syntactic informat...
1,The authors extend the out-of-distribution (OO...,The authors focus on large scale out-of-distri...,authors extend outofdistribution ood detection...,authors focus large scale outofdistribution oo...
2,This paper tries to give a measurement method ...,The problem studied in this paper is interesti...,paper tries give measurement method evaluate l...,problem studied paper interesting highlevel mo...
3,This paper proposes a new gradient attack meth...,The major criticism of this paper after the in...,paper proposes new gradient attack method name...,major criticism paper initial reviews lack exp...
4,This work proposed to evaluate the robustness ...,This paper proposes a novel challenge setting:...,work proposed evaluate robustness alphazero tr...,paper proposes novel challenge setting adversa...
...,...,...,...,...
1644,The paper generalises Shannon entropy-based ac...,The paper proposed a novel acquisition functio...,paper generalises shannon entropybased acquisi...,paper proposed novel acquisition function bo b...
1645,The paper studies the problem of value decompo...,The paper studies the join-Q value decompositi...,paper studies problem value decomposition deco...,paper studies joinq value decomposition proble...
1646,This paper aims to solve the degeneration prob...,"All four reviewers sided to accept the paper, ...",paper aims solve degeneration problem using co...,four reviewers sided accept paper proposed con...
1647,Verifying that one has the correct graph is an...,Meta Review: This paper proposes a test of a c...,verifying one correct graph important task sol...,meta review paper proposes test causal dag bas...


In [34]:
test_senten_tokenize = test_cleaned['input_tokenized'][0]

In [35]:
test_senten_tokenize_output = test_cleaned['Output'][0]

In [36]:
test_senten_tokenize_output

'The paper present results using syntactic information (primarily through constituency trees) on the task of recognizing argument discourse units. No reviewer recommends acceptance of the paper: - The empirical results appear strong, though the reviewers raise questions about some of the experimental choices.  - The writing is unclear and reviewers point out many missing or incorrect references in the bibliography. - There is little methodological novelty - known techniques are applied to a topic that has not been studied much. Overall, the area chair agrees with the reviewers that this work does not yet meet the bar for ICLR.'

In [37]:
# test_senten_tokenize

In [38]:
test_sentence = test_cleaned['Input'][0]

In [39]:
combined_input = prompt+test_sentence

In [40]:
device = next(model.parameters()).device 

# Testing the model to see how it is working

In [41]:
inputs = tokenizer(test_sentence, return_tensors='pt', max_length=512, truncation=True)

# Move the input tensors to the same device as the already-loaded model
inputs = {key: value.to(device) for key, value in inputs.items()}

# Generate the output based on the combined prompt and input
output_ids = model.generate(inputs['input_ids'], max_length= 1052, num_beams=5, early_stopping=True)

# Decode the output into human-readable text
output = tokenizer.decode(output_ids[0], skip_special_tokens=True)
print(output)

paper presents bert-gat-crf framework for recognizing argument units in sentences through constituency tree representation. examples of what is a consistency tree of short and long sequences are given. experiments on small datasets proved the performance of the bert-gat-crf algorithm.


In [42]:
# model_dir = '/kaggle/working/summarized_model'

# # Load the trained model
# model = AutoModelForSeq2SeqLM.from_pretrained(model_dir)

# # Load the tokenizer (if applicable)
# tokenizer = AutoTokenizer.from_pretrained(model_dir)

Adding the prompt to the input_tokenized column

In [43]:
test_cleaned["input_tokenized"] = test_cleaned["input_tokenized"].apply(lambda x: f"{prompt} : {x} ")

In [44]:
test_cleaned

,Input,Output,input_tokenized,output_tokenized
0,This paper presents a bert-gat-crf framework ...,The paper present results using syntactic info...,\n You are an assistant which helps i...,paper present results using syntactic informat...
1,The authors extend the out-of-distribution (OO...,The authors focus on large scale out-of-distri...,\n You are an assistant which helps i...,authors focus large scale outofdistribution oo...
2,This paper tries to give a measurement method ...,The problem studied in this paper is interesti...,\n You are an assistant which helps i...,problem studied paper interesting highlevel mo...
3,This paper proposes a new gradient attack meth...,The major criticism of this paper after the in...,\n You are an assistant which helps i...,major criticism paper initial reviews lack exp...
4,This work proposed to evaluate the robustness ...,This paper proposes a novel challenge setting:...,\n You are an assistant which helps i...,paper proposes novel challenge setting adversa...
...,...,...,...,...
1644,The paper generalises Shannon entropy-based ac...,The paper proposed a novel acquisition functio...,\n You are an assistant which helps i...,paper proposed novel acquisition function bo b...
1645,The paper studies the problem of value decompo...,The paper studies the join-Q value decompositi...,\n You are an assistant which helps i...,paper studies joinq value decomposition proble...
1646,This paper aims to solve the degeneration prob...,"All four reviewers sided to accept the paper, ...",\n You are an assistant which helps i...,four reviewers sided accept paper proposed con...
1647,Verifying that one has the correct graph is an...,Meta Review: This paper proposes a test of a c...,\n You are an assistant which helps i...,meta review paper proposes test causal dag bas...


Testing the model with prompt. The maximum length has been kept at 512 to save memory and runtime

In [45]:
# Install the required library if not already installed
!pip install -q evaluate
!pip install rouge_score

from evaluate import load

# Load the ROUGE metric using the new 'evaluate' library
rouge_metric = load('rouge')

import torch

# Function to generate summaries from the model
def generate_summary(model, tokenizer, inputs, max_length=512):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    # Move the model to the correct device
    model.to(device)
    
    # Tokenize the inputs and move them to the same device as the model
    inputs = tokenizer(inputs, return_tensors="pt", truncation=True, padding="max_length", max_length=512)
    inputs = {key: value.to(device) for key, value in inputs.items()}  # Move input tensors to the correct device
    
    # Generate summary using the model
    summaries = model.generate(input_ids=inputs['input_ids'], attention_mask=inputs['attention_mask'], max_length=max_length)
    
    # Decode and return the summary
    return tokenizer.decode(summaries[0], skip_special_tokens=True)


# Evaluate model on the original validation data (before tokenization)
def evaluate_model_on_original(model, tokenizer, original_validation_dataset):
    references = []
    predictions = []
    
    for i in range(len(original_validation_dataset)):
        input_text = original_validation_dataset.iloc[i]['input_tokenized']  # Input meta-review
        reference_summary = original_validation_dataset.iloc[i]['output_tokenized']  # Ground truth summary
        
        # Generate summary from the model
        predicted_summary = generate_summary(model, tokenizer, input_text)
        
        # Store the reference and prediction
        references.append(reference_summary)
        predictions.append(predicted_summary)
    
    # Compute ROUGE scores
    results = rouge_metric.compute(predictions=predictions, references=references)
    return results

# Example usage:
# Assuming you have your fine-tuned model and tokenizer
results = evaluate_model_on_original(model, tokenizer, test_cleaned)

# Display the ROUGE scores
print(f"ROUGE-1: {results['rouge1']}")
print(f"ROUGE-2: {results['rouge2']}")
print(f"ROUGE-L: {results['rougeL']}")




huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...

To disable this warning, you can either:

	- Avoid using `tokenizers` before the fork if possible

	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...

To disable this warning, you can either:

	- Avoid using `tokenizers` before the fork if possible

	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)











ROUGE-1: 0.12915540789053565

ROUGE-2: 0.0274730637117806

ROUGE-L: 0.09157599141429718
